In [3]:
data_file = r'C:\Users\matematik\Documents\lab\rbcarticles.json'

In [264]:
import json
import pandas as pd
import re
from datetime import datetime

# Перетворення у DataFrame
df = pd.DataFrame(data)

# Маппінг для заміни українських назв місяців і днів тижня на англійські
months = {
    'січня': 'January', 'лютого': 'February', 'березня': 'March',
    'квітня': 'April', 'травня': 'May', 'червня': 'June',
    'липня': 'July', 'серпня': 'August', 'вересня': 'September',
    'жовтня': 'October', 'листопада': 'November', 'грудня': 'December'
}

days = {
    'Понеділок': 'Monday', 'Вівторок': 'Tuesday', 'Середа': 'Wednesday',
    'Четвер': 'Thursday', 'П\'ятниця': 'Friday', 'Субота': 'Saturday', 'Неділя': 'Sunday'
}

# Список рядків, які потрібно видалити
text_to_remove = [' | РБК-Україна', " Раніше повідомлялось, що", "Нагадаємо, ми писали."]

# Список фраз, які потрібно видаляти з кінця тексту
end_phrases_to_remove = [
    'До речі, раніше ми писали, що',
    'Нагадаємо, раніше.',
    "Раніше повідомлялося,",
    "Раніше ми писали,",
    "А ще розповідали,",
    'Також ми писали,',
    "Також ми розповідали,"
    'Ми писали,',
    " Тим часом за результатами опитування стало відомо,",
    " Нагадаємо, ми писали, що.",
    " Нагадаємо, ми писали,",
    " Нагадаємо, ми писали.",
    " Термінові та важливі повідомлення про війну Росії проти України читайте на каналі РБК-Україна в.",
    "Термінові та важливі повідомлення про війну Росії проти України читайте на каналі РБК-Україна в."
    "Термінові та важливі повідомлення про війну Росії проти України читайте на каналі.",
    "Нагадаємо, раніше ми пояснювали,",
    "Також ми розповідали, і чому це важливо.",
    "Ми писали, Також повідомлялося.",
    "Раніше ми розповідали, чи зараховується Читайте також, які",
    "Ми писали, Також за результатами свіжого опитування стало відомо,",
    "До речі, раніше ми писали, А також розповідали, що",
    "Раніше ми повідомляли, що.",
    ", - читайте в",
    "Нагадаємо, у Також повідомлялося, що.",
    "Ми писали, Також повідомлялося, як переселенцям отримати",
    "Ми розповідали, Також повідомлялося.",
    "Також ми розповідали, що",
    "Докладніше про нові тарифи на ці напрямки –.",
    "Також ми розповідали.",
    "Нагадаємо, Також українка поділилася з РБК-Україна своєю думкою про те,",
    "РБК-Україна також писало, що.",
    "Термінові та важливі повідомлення про війну Росії проти України читайте на каналі.",
    "Нагадаємо, ми розповідали.",
    "Також ми розповідали."

]

# Функція для заміни українських назв на англійські
def replace_uk_to_en(date_str):
    for ua_month, en_month in months.items():
        date_str = date_str.replace(ua_month, en_month)
    for ua_day, en_day in days.items():
        date_str = date_str.replace(ua_day, en_day)
    return date_str

# Заміна назв місяців і днів та перетворення у datetime
df['article_datetime'] = df['article_datetime'].apply(replace_uk_to_en)
df['article_datetime'] = pd.to_datetime(df['article_datetime'], format='%A %d %B %Y %H:%M ')

# Функція для очищення тексту від технічних символів і видалення рядків
def clean_text(text):
    # Видалення екрануючих символів, зайвих пробілів, нових рядків і табуляцій
    text = re.sub(r'\\[tn]', ' ', text)
    text = re.sub(r'\s+', ' ', text).strip()


    # Видалення фраз
    for phrase in end_phrases_to_remove:
        text = text.replace(phrase, '')

    # Видалення вказаних рядків
    for t in text_to_remove:
        text = text.replace(t, '')
    # Видалення пробілів перед крапкою
    text = re.sub(r'\s+\.', '.', text)
    # Видалення пробілів перед комами
    text = re.sub(r'\s+,', ',', text)
    # Видалення повторюваних ком
    text = re.sub(r',+', ',', text)
    # Додавання пробілу після коми, якщо його немає
    text = re.sub(r',(\S)', r', \1', text)
    # Перетворення екранованих апострофів на звичайні
    text = re.sub(r"\\'", "'", text)
    # Заміна ", ." на крапку
    text = re.sub(r',\s*\.', '.', text)

    text = re.sub(rf'{re.escape("Також ми розповідали.")}$', '.', text)
    text = re.sub(rf'{re.escape("Зокрема.")}$', '.', text)
    text = re.sub(rf'{re.escape(" Також минулого тижня у Німеччині.")}$', '.', text)
    text = re.sub(rf'{re.escape("Термінові та важливі повідомлення про війну Росії проти України читайте на каналі.")}$', '.', text)
    text = re.sub(rf'{re.escape("Також ми повідомляли,")}$', '.', text)
    text = re.sub(rf'{re.escape(", де.")}$', '.', text)
    text = re.sub(rf'{re.escape("Нагадаємо, ми повідомляли, що.")}$', '.', text)
    text = re.sub(rf'{re.escape("чому .")}$', '.', text)
    text = re.sub(rf'{re.escape("Ми писали, ")}$', '.', text)
    text = re.sub(rf'{re.escape("Також ми розповідали,")}$', '.', text)
    text = re.sub(rf'{re.escape("  що Крім того,")}$', '.', text)
    text = re.sub(rf'{re.escape(".  що")}$', '.', text)
    text = re.sub(rf'{re.escape(".  що.")}$', '.', text)
    text = re.sub(rf'{re.escape(". що")}$', '.', text)
    text = re.sub(rf'{re.escape(". що.")}$', '.', text)
    text = re.sub(rf'{re.escape(",")}$', '.', text)
    text = re.sub(r'\.\s\.', ',', text)
    text = re.sub(r',+', ',', text)
    text = re.sub(r'\.\.+', '.', text)
    text = text.strip()
    text = re.sub(r',$', '.', text)



    return text

# Очистка тексту
df['article_title'] = df['article_title'].apply(clean_text)
df['article_text'] = df['article_text'].apply(clean_text)

df.sample(3).transpose()

,94,304,1306
article_url,https://www.rbc.ua/rus/travel/proyizniy-49-evr...,https://www.rbc.ua/rus/news/k-uchasniku-boyovi...,https://www.rbc.ua/rus/travel/wizz-air-zapusti...
article_uuid,6c12f15508860a7d5083afb7191abd5deba814b40475d1...,b7a9b0485c0676661dbcfa3b2fb70cb019313b45b454e8...,3149afb9f10900848935a0b221ce3089723fd074f7bd37...
article_id,1703608109,1703777169,1683893776
article_datetime,2023-12-27 11:22:00,2024-01-14 17:53:00,2023-05-12 15:16:00
article_title,Проїзний в Німеччині за 49 євро – чи діятиме у...,Пільги на комунальні послуги для учасників бой...,Wizz Air запустить нові маршрути з Польщі - ку...
article_text,Про це повідомляє з посиланням на німецькі ЗМІ...,"Учасники бойових дій мають право, зокрема, на ...",Бюджетна авіакомпанія Wizz Air відкриє 8 нових...


In [265]:
current_index = 15

def print_next_article(start_from=None):
    global current_index
    if start_from is not None:
        current_index = start_from

    if current_index < len(df):
        article_text = df.iloc[current_index].article_text
        print('Стаття №', current_index + 1)
        print(article_text)
        current_index += 1
    else:
        print("Більше статей немає.")


In [278]:
print_next_article()

Стаття № 28
Одним із критеріїв буде соціальна самодостатність, повідомляє з посиланням на. У Міністерстві внутрішніх справ Чехії готують систему, за яких умов українці зможуть залишатися у країні після закінчення дії тимчасового захисту. Одним із критеріїв буде соціальна самодостатніть, повідомив голова МВС Він планує найближчим часом презентувати ці умови урядовій робочій групі, яка розробляє плани адаптації й інтеграції біженців. Також планується провести переговори на цю тему з опозицією, частина якої налаштована проти прийому біженців із України. Раніше Віт Ракушан сказав, що у Чехії захоче залишитися 180 тисяч біженців. Такі підрахунки базуються на результатах опитування PAQ Research, за якими 58% українських біженців не планували повертатися до України з Чехії у найближчі два роки. Станом на початок березня 2024 року у Чехії було зареєстровано понад 386 тисяч осіб із тимчасовим захистом. За даними Бюро працевлаштування, наприкінці січня 2024 року працювали 117 тисяч українських б